This notebook contains code to create, train and test 2 layers FC on top of features from base models

In [ ]:
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.optimizers import RMSprop, SGD
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
from matplotlib import pyplot as plt
from PIL import Image
import random, csv, bson, io, tqdm, time
import pandas as pd
import numpy as np 
from scipy.misc import imread, imresize
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

num_classes = 5270

img_width = 180
img_height = 180

# 1. Create model

In [ ]:
with tf.device('/gpu:0'):
    model = Sequential()
    model.add(Dense(10540, input_shape=(21080,), activation = 'relu'))
    model.add(Dense(5270, activation = 'softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

# 2. Preparations and utils

Compile models

In [ ]:
with tf.device('/gpu:1'):
    model_single = load_model('models/74.hdf5', custom_objects={'tf':tf})

with tf.device('/gpu:2'):
    model_additional = load_model('models/85.h5', custom_objects={'tf':tf})

Read categories and create decoder for one-hot encoding

In [ ]:
df = pd.read_csv('data/category_names.csv') 
categories_list = df.category_id

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(categories_list)
encoded_Y = encoder.transform(categories_list)

categories_one_hot = to_categorical(encoded_Y)

In [ ]:
def get_image_new(pic):
    img = load_img(io.BytesIO(pic['picture']), target_size = (180, 180))
    x = img_to_array(img)
    img = preprocess_input(x)
    return img

# 3. Getting base predictions and training model

In [ ]:
batch_size = 256

In [ ]:
def predict(single_data, additional_data, additional_images, categories):
    result = []
    num_single = len(single_data)
    num_additional = len(additional_data)

    datagen = ImageDataGenerator(
    )
    
    single_train_generator = datagen.flow(
        single_data,
        batch_size=batch_size,
        shuffle = False,
        )
    
    additional_train_generator = datagen.flow(
        single_data,
        batch_size=batch_size,
        shuffle = False,
        )

    #print("Generating batch  --- %s seconds ---" % (time.time() - start_time))
    #start_time = time.time()
    
    probabilities_additional = model_additional.predict_generator(additional_train_generator,
                                                                   steps = num_additional//batch_size,
                                                                   workers = 16)
    #predicting on single model (first image)
    probabilities_single = model_single.predict_generator(single_train_generator, 
                                                           steps = num_single//batch_size,
                                                           workers = 16)
    #print("Prediciting --- %s seconds ---" % (time.time() - start_time))
    #start_time = time.time()            

    #compose resulting data frame
    for i in range(n):
        probabilities_additional_i_flatten = np.empty(0)
        if additional_images[i + 1] - additional_images[i] != 0:
            probabilities_additional_i_flatten = probabilities_additional[additional_images[i]:additional_images[i + 1],:].flatten()
        probabilities = []

        probabilities = probabilities_single[i,:].tolist()
        if probabilities_additional_i_flatten.shape[0] != 0:
            probabilities = probabilities + probabilities_additional_i_flatten.tolist()
        # adding zeros at the end of every row if we have less than 4 photos
        zeros = [0] * (4* num_classes - len(probabilities))
        probabilities =  probabilities + zeros

        assert len(probabilities) == 4* num_classes
        #print(len(probabilities))
        result.append(probabilities)
    #print("Rearenging results --- %s seconds ---" % (time.time() - start_time))
    #start_time = time.time()
    return result

In [ ]:
with open('./data/train.bson', 'rb') as fbson:
    data = bson.decode_file_iter(fbson)
    single = []
    additional = []
    additional_images = {}
    additional_images[0] = 0
    categories = []
    batch = []
    m = 0
    n = 0 # product index in a batch
    k = 0 # additional_images index in a batch
    start_time = time.time()
    for c, d in enumerate(data):  
        _id = d['_id']
        category = d['category_id']
        pics = d['imgs']
        categories.append(category)
        
        for e, pic in enumerate(d['imgs']):
            if e == 0:
                single.append(get_image_new(pic))
            else:
                additional.append(get_image_new(pic))
                k += 1
        #print("First --- %s seconds ---" % (time.time() - start_time))
        #start_time = time.time()
        additional_images[n + 1] = k
        n += 1
        #print("Second --- %s seconds ---" % (time.time() - start_time))
        if n == batch_size*20:
            result = predict(np.array(single), np.array(additional), additional_images, categories)
            #encode Y
            encoded_Y = encoder.transform(categories)
            categories_one_hot = to_categorical(encoded_Y, num_classes = 5270)

            #train 2nd layer model
            #print(m)
            #if m == 7:
            #    #print ("m == 0")
            #    print(model.test_on_batch(np.array(result), categories_one_hot))
            #    m = 0
            #model.train_on_batch(np.array(result), categories_one_hot)

            #print("Model traing --- %s seconds ---" % (time.time() - start_time))
            #start_time = time.time()

            k = 0
            n = 0
            additional_images = {}
            additional_images[0] = 0
            single = []
            additional = []
            result = []
            categories = []

        if c % 14000 == 0:
            #print("Generating batch  --- %s seconds ---" % (time.time() - start_time))
            #start_time = time.time()
            print(str(c) + " products done. " + str(c/7000000*100) + "%%. --- %s seconds ---" % (time.time() - start_time))

In [ ]:
model.save("models/00_1410000.h5")

# 4. Testing

In [ ]:
model = load_model('models/00_560000.h5', custom_objects={'tf':tf})

In [ ]:
with open('./data/test.bson', 'rb') as fbson:
    data = bson.decode_file_iter(fbson)
    single = []
    additional = []
    
    additional_images = {}
    additional_images[0] = 0

    categories = []
    result = []
    batch = []
    ids = []
    
    final_predictions = []
    
    n = 0 # product index in a batch
    k = 0 # additional_images index in a batch
    start_time = time.time()
    for c, d in enumerate(data):  
        _id = d['_id']
        pics = d['imgs']
        ids.append(_id)
        
        for e, pic in enumerate(d['imgs']):
            if e == 0:
                single.append(get_image_new(pic))
            else:
                additional.append(get_image_new(pic))
                k += 1
        #print("First --- %s seconds ---" % (time.time() - start_time))
        #start_time = time.time()

        #single.append(img_0)
        #np.concatenate((additional, batch), axis = 0)
        #batch = np.empty((0,180,180,3))

        additional_images[n + 1] = k
        n += 1
        #print("Second --- %s seconds ---" % (time.time() - start_time))
          if n == batch_size*20:
            result = predict(np.array(single), np.array(additional), additional_images, categories)
            #encode Y
            encoded_Y = encoder.transform(categories)
            categories_one_hot = to_categorical(encoded_Y, num_classes = 5270)

            #print(encoded_res)
            final_predictions += encoder.inverse_transform(encoded_res).tolist()
            
            #print("Model traing --- %s seconds ---" % (time.time() - start_time))
            #start_time = time.time()
            
            k = 0
            n = 0
            additional_images = {}
            additional_images[0] = 0
            single = []
            additional = []
            result = []
            categories = []

        if c % 1768 == 0:
            print(str(c) + " products done. " + str(c/1768182*100) + "%%. --- %s seconds ---" % (time.time() - start_time))
if n != 0:
    #start_time = time.time()
    #predicting on additional model (using all images except for first one)
    #print("additional_images" + str(additional_images))
    probabilities_additional = np.array(model_additional.predict(np.array(additional), batch_size = 100))
    #predicting on single model (first image)
    probabilities_single = np.array(model_single.predict(np.array(single), batch_size = 100))

    #print("Prediciting --- %s seconds ---" % (time.time() - start_time))
    #start_time = time.time()            

    #compose resulting data frame
    for i in range(n):
        probabilities_additional_i_flatten = np.empty(0)
        #print("Single - " + str(np.argmax(probabilities_single[i,:])))
        if additional_images[i + 1] - additional_images[i] != 0:
            #print(probabilities_additional[additional_images[i]:additional_images[i + 1],:])
            #print("Additional - " + str(np.argmax(probabilities_additional[additional_images[i],:])))
            probabilities_additional_i_flatten = probabilities_additional[additional_images[i]:additional_images[i + 1],:].flatten()
        probabilities = []

        #print("Actual - " + str(category))
        #print("len probabilities_additional_i_flatten" + str(len(probabilities_additional_i_flatten.tolist())))
        probabilities = probabilities_single[i,:].tolist()
        if probabilities_additional_i_flatten.shape[0] != 0:
            probabilities = probabilities + probabilities_additional_i_flatten.tolist()
        # adding zeros at the end of every row if we have less than 4 photos
        zeros = [0] * (4* num_classes - len(probabilities))
        probabilities =  probabilities + zeros

        assert len(probabilities) == 4* num_classes
        #print(len(probabilities))
        result.append(probabilities)
    #print("Rearenging results --- %s seconds ---" % (time.time() - start_time))
    #start_time = time.time()

    prob = model.predict_on_batch(np.array(result))
    encoded_res = np.argmax(prob, axis = 1)
    #print(encoded_res)
    final_predictions += encoder.inverse_transform(encoded_res).tolist()            

to_save = pd.DataFrame(np.column_stack((ids, final_predictions)), dtype = 'int32')
to_save.columns = ['_id', 'category_id']
to_save.to_csv("result.csv", index = False)
print("saved")